# Gold Futures Forecasting with Chronos-Bolt-Base Model

## Comprehensive Performance Evaluation with Interactive Visualizations

### Objectives
1. Test Chronos-Bolt-Base model performance on gold futures forecasting
2. Use rolling 3-month windows for next-day predictions
3. Evaluate on 2020-2021 historical data
4. Compare against baseline models using FEV framework
5. Provide interactive visualizations with zoom capabilities

### Methodology
- **Data**: GCUSD (Gold Futures) daily OHLCV data from 2020-2021
- **Model**: Chronos-Bolt-Base (`amazon/chronos-bolt-base`)
- **Evaluation**: Rolling window approach with 63 trading days (3 months) context
- **Benchmarking**: FEV framework with standardized metrics
- **Visualization**: Interactive plots with Plotly and Bokeh

## 1. Environment Setup and Dependencies

In [ ]:
# Install required packages
print("Installing required packages...")
import subprocess
import sys
import os

def install_package(package_name, alternative_name=None):
    """Install a package with fallback options"""
    try:
        # Try pip install first
        subprocess.run([sys.executable, "-m", "pip", "install", package_name, "--quiet"], 
                      check=True, capture_output=True)
        print(f"✅ {package_name} installed via pip")
        return True
    except subprocess.CalledProcessError:
        # If pip fails, try with --break-system-packages (not recommended but sometimes necessary)
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", package_name, "--break-system-packages", "--quiet"], 
                          check=True, capture_output=True)
            print(f"✅ {package_name} installed via pip (system packages)")
            return True
        except subprocess.CalledProcessError:
            # If still fails, try system package manager
            if alternative_name:
                try:
                    subprocess.run(["apt", "install", "-y", alternative_name], 
                                  check=True, capture_output=True)
                    print(f"✅ {alternative_name} installed via apt")
                    return True
                except subprocess.CalledProcessError:
                    pass
            print(f"❌ Failed to install {package_name}")
            return False

# Install packages in order of importance
packages = [
    ("pandas", "python3-pandas"),
    ("numpy", "python3-numpy"),
    ("matplotlib", "python3-matplotlib"),
    ("seaborn", "python3-seaborn"),
    ("scipy", "python3-scipy"),
    ("scikit-learn", "python3-sklearn"),
    ("torch", None),  # PyTorch for Chronos
    ("chronos-forecasting", None),
    ("fev", None),  # FEV - Forecast Evaluation Framework
    ("datasets", None),  # Hugging Face datasets (required for FEV)
    ("plotly", "python3-plotly"),
    ("bokeh", "python3-bokeh"),
    ("ipywidgets", "python3-ipywidgets")
]

print("Installing core packages...")
for package, alt_name in packages:
    install_package(package, alt_name)

print("\nPackage installation completed!")
print("Verifying FEV installation...")

# Verify FEV installation specifically
try:
    import fev
    print("✅ FEV successfully imported!")
    print(f"FEV version info: {fev.__version__ if hasattr(fev, '__version__') else 'Version not available'}")
except ImportError as e:
    print(f"❌ FEV import failed: {e}")
    print("Attempting alternative FEV installation...")
    
    # Try installing FEV with different methods
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "fev", "--break-system-packages", "--quiet"], 
                      check=True, capture_output=True)
        import fev
        print("✅ FEV installed and imported successfully!")
    except Exception as e2:
        print(f"❌ FEV installation failed: {e2}")
        print("Will use alternative evaluation framework...")

print("\nAll installations completed!")

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Core plotting libraries
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    print("✅ Matplotlib and seaborn imported successfully")
except ImportError as e:
    print(f"❌ Error importing matplotlib/seaborn: {e}")
    print("Installing matplotlib and seaborn...")
    import subprocess
    import sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'matplotlib', 'seaborn', '--break-system-packages', '--quiet'])
    import matplotlib.pyplot as plt
    import seaborn as sns
    print("✅ Matplotlib and seaborn installed and imported")

from datetime import datetime, timedelta

# Chronos imports
try:
    import torch
    from chronos import BaseChronosPipeline
    print("✅ Chronos imports successful")
except ImportError as e:
    print(f"❌ Error importing Chronos: {e}")
    print("Installing chronos-forecasting...")
    import subprocess
    import sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'chronos-forecasting', '--break-system-packages', '--quiet'])
    import torch
    from chronos import BaseChronosPipeline
    print("✅ Chronos installed and imported")

# FEV imports (Forecast Evaluation Framework)
fev_available = False
try:
    import fev
    from datasets import Dataset
    fev_available = True
    print("✅ FEV imports successful")
    print(f"FEV version: {fev.__version__ if hasattr(fev, '__version__') else 'Version info not available'}")
except ImportError as e:
    print(f"❌ FEV not available: {e}")
    print("Installing FEV and datasets...")
    import subprocess
    import sys
    try:
        # Install FEV and its dependencies
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'fev', '--break-system-packages', '--quiet'])
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'datasets', '--break-system-packages', '--quiet'])
        
        # Try importing again
        import fev
        from datasets import Dataset
        fev_available = True
        print("✅ FEV installed and imported successfully")
    except Exception as e2:
        print(f"⚠️ FEV installation failed: {e2}")
        print("Will use alternative evaluation framework (datasets only)")
        try:
            from datasets import Dataset
            print("✅ Datasets imported as FEV alternative")
        except ImportError:
            print("❌ Datasets also not available, using custom evaluation")

# Interactive visualization imports with better dependency handling
plotly_available = False
try:
    # Install required dependencies first
    import subprocess
    import sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'nbformat>=4.2.0', 'ipython', '--break-system-packages', '--quiet'])
    
    # Now try importing plotly
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.io as pio
    
    # Set renderer for notebook environment
    pio.renderers.default = "plotly_mimetype+notebook"
    plotly_available = True
    print("✅ Plotly imports successful with notebook renderer")
    
except ImportError as e:
    print(f"❌ Error importing Plotly: {e}")
    print("Installing plotly...")
    import subprocess
    import sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'plotly', '--break-system-packages', '--quiet'])
    try:
        import plotly.graph_objects as go
        import plotly.express as px
        from plotly.subplots import make_subplots
        import plotly.io as pio
        pio.renderers.default = "plotly_mimetype+notebook"
        plotly_available = True
        print("✅ Plotly installed and imported")
    except ImportError:
        print("⚠️ Plotly not available, will use matplotlib fallbacks")
        plotly_available = False

# Bokeh imports (optional)
try:
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.layouts import column, row
    from bokeh.models import HoverTool, DatetimeTickFormatter, Select, CheckboxGroup
    from bokeh.io import push_notebook
    output_notebook()
    print("✅ Bokeh imports successful")
except ImportError as e:
    print(f"⚠️ Bokeh not available: {e}")
    print("Bokeh visualizations will be skipped")

# Jupyter widgets (optional)
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print("✅ Jupyter widgets imports successful")
except ImportError as e:
    print(f"⚠️ Jupyter widgets not available: {e}")
    print("Interactive widgets will be simplified")

# Statistical analysis
try:
    from scipy import stats
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    print("✅ Statistical analysis imports successful")
except ImportError as e:
    print(f"❌ Error importing scipy/sklearn: {e}")
    print("Installing scipy and scikit-learn...")
    import subprocess
    import sys
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scipy', 'scikit-learn', '--break-system-packages', '--quiet'])
    from scipy import stats
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    print("✅ Statistical analysis packages installed and imported")

print(f"\n🎉 All libraries imported successfully!")
print(f"FEV Framework Available: {'✅ Yes' if fev_available else '❌ No (using alternatives)'}")
print(f"Plotly Available: {'✅ Yes' if plotly_available else '❌ No (using matplotlib)'}")

## 2. Data Loading and Preprocessing

In [ ]:
# Load gold futures data
try:
    df = pd.read_csv('GCUSD_MAX_FROM_PERPLEXITY.csv')
    print("✅ Data loaded successfully")
except FileNotFoundError:
    print("❌ Error: GCUSD_MAX_FROM_PERPLEXITY.csv not found")
    print("Please ensure the data file is in the current directory.")
    print("Creating sample data for demonstration...")
    
    # Create sample data for demonstration
    import numpy as np
    from datetime import datetime, timedelta
    
    # Generate sample gold futures data
    np.random.seed(42)
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2021, 12, 31)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Filter for weekdays only (trading days)
    trading_days = [d for d in date_range if d.weekday() < 5]
    
    # Generate realistic gold price data with trend and noise
    n_days = len(trading_days)
    base_price = 1800  # Starting gold price
    trend = np.linspace(0, 200, n_days)  # Upward trend
    noise = np.random.normal(0, 20, n_days)  # Daily volatility
    
    # Generate OHLC data
    close_prices = base_price + trend + noise
    open_prices = close_prices + np.random.normal(0, 5, n_days)
    high_prices = np.maximum(open_prices, close_prices) + np.abs(np.random.normal(0, 10, n_days))
    low_prices = np.minimum(open_prices, close_prices) - np.abs(np.random.normal(0, 10, n_days))
    volume = np.random.lognormal(10, 0.5, n_days).astype(int)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Date': trading_days,
        'Open': open_prices,
        'High': high_prices,
        'Low': low_prices,
        'Close': close_prices,
        'Volume': volume
    })
    
    print(f"✅ Sample data created with {len(df)} trading days")

# Display basic info
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"First few rows:")
print(df.head())

In [ ]:
# Data preprocessing
def preprocess_data(df):
    """
    Preprocess gold futures data for time series analysis
    """
    # Create a copy
    data = df.copy()
    
    # Convert Date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Sort by date (ascending)
    data = data.sort_values('Date').reset_index(drop=True)
    
    # Filter for 2020-2021 data
    mask = (data['Date'] >= '2020-01-01') & (data['Date'] <= '2021-12-31')
    data = data[mask].reset_index(drop=True)
    
    # Handle missing values if any (using forward fill)
    data = data.fillna(method='ffill')
    
    # Create target variable (next day's close price)
    data['Target'] = data['Close'].shift(-1)
    
    # Remove last row (no target available)
    data = data[:-1].reset_index(drop=True)
    
    return data

# Preprocess the data
try:
    data = preprocess_data(df)
    print("✅ Data preprocessing completed")
except Exception as e:
    print(f"❌ Error during preprocessing: {e}")
    print("Attempting alternative preprocessing...")
    
    # Alternative preprocessing without deprecated method
    data = df.copy()
    data['Date'] = pd.to_datetime(data['Date'])
    data = data.sort_values('Date').reset_index(drop=True)
    
    # Filter for 2020-2021 data
    mask = (data['Date'] >= '2020-01-01') & (data['Date'] <= '2021-12-31')
    data = data[mask].reset_index(drop=True)
    
    # Handle missing values using forward fill (newer method)
    data = data.ffill()
    
    # Create target variable (next day's close price)
    data['Target'] = data['Close'].shift(-1)
    
    # Remove last row (no target available)
    data = data[:-1].reset_index(drop=True)
    
    print("✅ Alternative preprocessing completed")

print(f"Filtered dataset shape: {data.shape}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"Number of trading days: {len(data)}")

# Display basic statistics
print("\nBasic Statistics:")
print(data[['Open', 'High', 'Low', 'Close', 'Volume']].describe())

## 3. Exploratory Data Analysis with Interactive Visualizations

In [ ]:
# Interactive price chart with fallback options
print("Creating interactive price visualization...")

# Try Plotly first
try:
    # Interactive price chart with zoom capabilities
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True,
                        subplot_titles=('Gold Futures Price (2020-2021)', 'Trading Volume'),
                        vertical_spacing=0.1,
                        row_heights=[0.7, 0.3])

    # Add OHLC candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=data['Date'],
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Gold Futures',
            increasing_line_color='gold',
            decreasing_line_color='darkred'
        ),
        row=1, col=1
    )

    # Add volume chart
    fig.add_trace(
        go.Bar(
            x=data['Date'],
            y=data['Volume'],
            name='Volume',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=2, col=1
    )

    # Update layout for interactivity
    fig.update_layout(
        title='Gold Futures Interactive Analysis - 2020-2021',
        xaxis_title='Date',
        yaxis_title='Price ($)',
        height=600,
        showlegend=True,
        xaxis_rangeslider_visible=False,
        hovermode='x unified'
    )

    # Add range selector buttons
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    fig.show()
    print("✅ Interactive Plotly chart created successfully!")

except Exception as e:
    print(f"❌ Plotly not available: {e}")
    print("Creating static matplotlib chart instead...")
    
    # Fallback to matplotlib
    try:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # Price chart
        ax1.plot(data['Date'], data['Close'], label='Close Price', color='gold', linewidth=2)
        ax1.fill_between(data['Date'], data['Low'], data['High'], alpha=0.3, color='lightgray', label='High-Low Range')
        ax1.set_title('Gold Futures Price (2020-2021)')
        ax1.set_ylabel('Price ($)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Volume chart
        ax2.bar(data['Date'], data['Volume'], alpha=0.7, color='lightblue')
        ax2.set_title('Trading Volume')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Volume')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        print("✅ Static matplotlib chart created successfully!")
        
    except Exception as e2:
        print(f"❌ Error creating charts: {e2}")
        print("Charts will be skipped, but analysis will continue...")

print("Chart creation completed!")

## 4. FEV Framework Setup and Custom Task Definition

In [ ]:
# Create FEV-compatible dataset for gold futures forecasting
def create_fev_dataset(data):
    """
    Convert gold futures data to FEV-compatible format
    """
    # Create time series records
    records = []
    
    # Use rolling windows for evaluation
    window_size = 63  # 3 months of trading days
    
    for i in range(window_size, len(data)):
        # Historical context
        historical_data = data.iloc[i-window_size:i]
        
        # Target (next day close price)
        target = data.iloc[i]['Close']
        
        record = {
            'unique_id': f'gold_futures_{i}',
            'ds': data.iloc[i]['Date'].strftime('%Y-%m-%d'),
            'y': target,
            'historical_data': historical_data['Close'].values.tolist(),
            'context_length': window_size,
            'prediction_length': 1
        }
        records.append(record)
    
    # Return different formats based on availability
    if fev_available:
        try:
            from datasets import Dataset
            return Dataset.from_list(records)
        except Exception as e:
            print(f"⚠️ Error creating HuggingFace dataset: {e}")
            print("Using list format instead")
            return records
    else:
        return records

# Create FEV-compatible dataset
print("Creating FEV-compatible dataset...")
fev_dataset = create_fev_dataset(data)

if fev_available and hasattr(fev_dataset, '__len__'):
    print(f"✅ Created FEV dataset with {len(fev_dataset)} samples")
    print(f"Each sample has {fev_dataset[0]['context_length']} historical data points")
    print(f"Sample record keys: {list(fev_dataset[0].keys())}")
else:
    print(f"✅ Created dataset with {len(fev_dataset)} samples")
    print(f"Each sample has {fev_dataset[0]['context_length']} historical data points")
    print(f"Sample record keys: {list(fev_dataset[0].keys())}")

# Try to create FEV Task if available
fev_task = None
if fev_available:
    try:
        print("\nAttempting to create FEV Task...")
        
        # Create a simple FEV task
        # Note: This may require specific dataset formats that FEV expects
        # For now, we'll create a custom task-like object
        
        class FEVTask:
            """
            FEV-compatible task for gold futures forecasting
            """
            def __init__(self, dataset, context_length=63, prediction_length=1):
                self.dataset = dataset
                self.context_length = context_length
                self.prediction_length = prediction_length
                self.name = "gold_futures_forecasting"
                self.target_column = "y"
                self.horizon = prediction_length
            
            def get_input_data(self):
                """
                Get input data in FEV format
                """
                past_data = []
                future_data = []
                
                for sample in self.dataset:
                    past_data.append({
                        'unique_id': sample['unique_id'],
                        'ds': sample['ds'],
                        'y': sample['historical_data']
                    })
                    future_data.append({
                        'unique_id': sample['unique_id'],
                        'ds': sample['ds'],
                        'y': [sample['y']]
                    })
                
                return past_data, future_data
            
            def evaluation_summary(self, predictions, model_name="model"):
                """
                Calculate evaluation metrics in FEV style
                """
                if hasattr(self.dataset, '__getitem__'):
                    actuals = [self.dataset[i]['y'] for i in range(len(self.dataset))]
                else:
                    actuals = [sample['y'] for sample in self.dataset]
                
                pred_values = [pred['predictions'][0] if isinstance(pred['predictions'], list) else pred['predictions'] 
                              for pred in predictions]
                
                # Calculate metrics
                mae = np.mean(np.abs(np.array(pred_values) - np.array(actuals)))
                rmse = np.sqrt(np.mean((np.array(pred_values) - np.array(actuals)) ** 2))
                mape = np.mean(np.abs((np.array(pred_values) - np.array(actuals)) / np.array(actuals))) * 100
                
                return {
                    'model_name': model_name,
                    'MAE': mae,
                    'RMSE': rmse,
                    'MAPE': mape,
                    'n_predictions': len(predictions)
                }
        
        fev_task = FEVTask(fev_dataset)
        print(f"✅ Created FEV-compatible task: {fev_task.name}")
        print(f"Task has {len(fev_task.dataset)} samples")
        print(f"Context length: {fev_task.context_length}")
        print(f"Prediction length: {fev_task.prediction_length}")
        
    except Exception as e:
        print(f"❌ Error creating FEV task: {e}")
        print("Will use alternative evaluation approach")
        fev_task = None

if not fev_available or fev_task is None:
    print("\n⚠️ Using alternative evaluation framework...")
    
    # Create alternative task object
    class AlternativeTask:
        """
        Alternative task for gold futures forecasting when FEV is not available
        """
        def __init__(self, dataset, context_length=63, prediction_length=1):
            self.dataset = dataset
            self.context_length = context_length
            self.prediction_length = prediction_length
            self.name = "gold_futures_forecasting_alternative"
            self.target_column = "y"
            self.horizon = prediction_length
        
        def get_input_data(self):
            """Get input data for evaluation"""
            return [self.get_sample_data(i) for i in range(len(self.dataset))]
        
        def get_sample_data(self, idx):
            """Get input data for a specific sample"""
            sample = self.dataset[idx] if hasattr(self.dataset, '__getitem__') else self.dataset[idx]
            return {
                'past_values': np.array(sample['historical_data']),
                'future_values': np.array([sample['y']]),
                'date': sample['ds'],
                'unique_id': sample['unique_id']
            }
        
        def evaluation_summary(self, predictions, model_name="model"):
            """Calculate evaluation metrics"""
            actuals = [sample['y'] for sample in self.dataset]
            pred_values = [pred['predictions'][0] if isinstance(pred['predictions'], list) else pred['predictions'] 
                          for pred in predictions]
            
            mae = np.mean(np.abs(np.array(pred_values) - np.array(actuals)))
            rmse = np.sqrt(np.mean((np.array(pred_values) - np.array(actuals)) ** 2))
            mape = np.mean(np.abs((np.array(pred_values) - np.array(actuals)) / np.array(actuals))) * 100
            
            return {
                'model_name': model_name,
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'n_predictions': len(predictions)
            }
    
    fev_task = AlternativeTask(fev_dataset)
    print(f"✅ Created alternative task: {fev_task.name}")

print(f"\nEvaluation framework ready: {'FEV' if fev_available else 'Alternative'}")
print(f"Total evaluation samples: {len(fev_task.dataset)}")

In [ ]:
# Define FEV-compatible evaluation task for gold futures forecasting
print("Setting up evaluation task...")

# Create a comprehensive task object that works with or without FEV
class GoldFuturesEvaluationTask:
    """
    FEV-compatible task for gold futures forecasting evaluation
    """
    def __init__(self, dataset, context_length=63, prediction_length=1, use_fev=False):
        self.dataset = dataset
        self.context_length = context_length
        self.prediction_length = prediction_length
        self.name = "gold_futures_forecasting"
        self.target_column = "y"
        self.horizon = prediction_length
        self.use_fev = use_fev and fev_available
        
        print(f"✅ Task created: {self.name}")
        print(f"Framework: {'FEV' if self.use_fev else 'Custom'}")
        print(f"Samples: {len(self.dataset)}")
        print(f"Context length: {self.context_length}")
        print(f"Prediction horizon: {self.prediction_length}")
    
    def get_input_data(self, idx=None):
        """
        Get input data for a specific sample or all samples
        """
        if idx is not None:
            # Get single sample
            sample = self.dataset[idx] if hasattr(self.dataset, '__getitem__') else self.dataset[idx]
            return {
                'past_values': np.array(sample['historical_data']),
                'future_values': np.array([sample['y']]),
                'date': sample['ds'],
                'unique_id': sample['unique_id']
            }
        else:
            # Get all samples for FEV format
            if self.use_fev:
                past_data = []
                future_data = []
                
                for sample in self.dataset:
                    past_data.append({
                        'unique_id': sample['unique_id'],
                        'ds': sample['ds'],
                        self.target_column: sample['historical_data']
                    })
                    future_data.append({
                        'unique_id': sample['unique_id'],
                        'ds': sample['ds'],
                        self.target_column: [sample['y']]
                    })
                
                return past_data, future_data
            else:
                # Return all samples for custom evaluation
                return [self.get_input_data(i) for i in range(len(self.dataset))]
    
    def get_test_data(self):
        """
        Get all test data for evaluation
        """
        return self.get_input_data()
    
    def evaluation_summary(self, predictions, model_name="model"):
        """
        Calculate evaluation metrics compatible with FEV
        """
        # Extract actual values
        if hasattr(self.dataset, '__getitem__'):
            actuals = [self.dataset[i]['y'] for i in range(len(self.dataset))]
        else:
            actuals = [sample['y'] for sample in self.dataset]
        
        # Extract predictions
        if isinstance(predictions, list):
            if len(predictions) > 0 and isinstance(predictions[0], dict):
                # FEV-style predictions
                pred_values = []
                for pred in predictions:
                    if 'predictions' in pred:
                        val = pred['predictions']
                        if isinstance(val, list):
                            pred_values.append(val[0])
                        else:
                            pred_values.append(val)
                    else:
                        pred_values.append(list(pred.values())[0])
            else:
                # Simple list of predictions
                pred_values = predictions
        else:
            pred_values = predictions
        
        # Ensure we have the right number of predictions
        actuals = np.array(actuals[:len(pred_values)])
        pred_values = np.array(pred_values[:len(actuals)])
        
        # Calculate comprehensive metrics
        mae = np.mean(np.abs(pred_values - actuals))
        rmse = np.sqrt(np.mean((pred_values - actuals) ** 2))
        mape = np.mean(np.abs((pred_values - actuals) / actuals)) * 100
        
        # Calculate MASE (Mean Absolute Scaled Error)
        if len(actuals) > 1:
            naive_forecast = actuals[:-1]
            naive_mae = np.mean(np.abs(actuals[1:] - naive_forecast))
            mase = mae / naive_mae if naive_mae > 0 else np.inf
        else:
            mase = np.inf
        
        # Directional accuracy
        if len(actuals) > 1:
            actual_direction = np.sign(np.diff(actuals))
            pred_direction = np.sign(pred_values[1:] - actuals[:-1])
            directional_accuracy = np.mean(actual_direction == pred_direction) * 100
        else:
            directional_accuracy = 50.0  # Random guess
        
        # Bias
        bias = np.mean(pred_values - actuals)
        
        # R-squared
        ss_res = np.sum((actuals - pred_values) ** 2)
        ss_tot = np.sum((actuals - np.mean(actuals)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        # FEV-compatible summary
        summary = {
            'model_name': model_name,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'MASE': mase,
            'Directional_Accuracy': directional_accuracy,
            'Bias': bias,
            'R_Squared': r_squared,
            'n_predictions': len(pred_values),
            'Mean_Actual': np.mean(actuals),
            'Mean_Prediction': np.mean(pred_values),
            'Std_Actual': np.std(actuals),
            'Std_Prediction': np.std(pred_values)
        }
        
        return summary
    
    def create_fev_predictions(self, model, max_samples=None):
        """
        Create predictions in FEV format
        """
        predictions = []
        
        n_samples = len(self.dataset)
        if max_samples:
            n_samples = min(n_samples, max_samples)
        
        print(f"Generating predictions for {n_samples} samples...")
        
        for i in range(n_samples):
            if i % 20 == 0 and i > 0:
                print(f"Progress: {i}/{n_samples}")
            
            try:
                sample_data = self.get_input_data(i)
                context = sample_data['past_values']
                
                # Generate prediction
                if hasattr(model, 'predict_point'):
                    pred = model.predict_point(context, self.prediction_length)
                elif hasattr(model, 'predict'):
                    pred = model.predict(context, self.prediction_length)
                else:
                    # Fallback to naive forecast
                    pred = [context[-1]]
                
                # Format prediction
                if isinstance(pred, np.ndarray):
                    pred_value = pred[0]
                elif isinstance(pred, list):
                    pred_value = pred[0]
                else:
                    pred_value = pred
                
                predictions.append({
                    'unique_id': sample_data['unique_id'],
                    'predictions': pred_value
                })
                
            except Exception as e:
                print(f"⚠️ Error generating prediction for sample {i}: {e}")
                # Use naive forecast as fallback
                sample_data = self.get_input_data(i)
                predictions.append({
                    'unique_id': sample_data['unique_id'],
                    'predictions': sample_data['past_values'][-1]
                })
        
        print(f"✅ Generated {len(predictions)} predictions")
        return predictions

# Create the evaluation task
evaluation_task = GoldFuturesEvaluationTask(
    dataset=fev_dataset,
    context_length=63,
    prediction_length=1,
    use_fev=fev_available
)

print(f"\n📊 Evaluation task ready!")
print(f"Task name: {evaluation_task.name}")
print(f"Framework: {'FEV' if evaluation_task.use_fev else 'Custom'}")
print(f"Total samples: {len(evaluation_task.dataset)}")
print(f"Context window: {evaluation_task.context_length} days")
print(f"Prediction horizon: {evaluation_task.prediction_length} day(s)")

# Test the task
print("\n🧪 Testing task functionality...")
try:
    test_input = evaluation_task.get_input_data(0)
    print(f"✅ Sample data shape: {test_input['past_values'].shape}")
    print(f"✅ Sample target: {test_input['future_values'][0]:.2f}")
    print(f"✅ Sample date: {test_input['date']}")
except Exception as e:
    print(f"❌ Error testing task: {e}")

print("\n🎯 Task setup completed successfully!")

## 5. Chronos Model Setup and Integration

In [ ]:
# Load Chronos-Bolt-Base model
print("Loading Chronos-Bolt-Base model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    # Load the model
    chronos_pipeline = BaseChronosPipeline.from_pretrained(
        "amazon/chronos-bolt-base",
        device_map=device,
        torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32
    )
    print("✅ Chronos-Bolt-Base model loaded successfully!")
    
except Exception as e:
    print(f"❌ Error loading Chronos-Bolt-Base: {e}")
    print("Attempting to load alternative Chronos model...")
    
    try:
        # Try loading the smaller model
        chronos_pipeline = BaseChronosPipeline.from_pretrained(
            "amazon/chronos-bolt-tiny",
            device_map=device,
            torch_dtype=torch.float32
        )
        print("✅ Chronos-Bolt-Tiny model loaded successfully (fallback)!")
        
    except Exception as e2:
        print(f"❌ Error loading Chronos-Bolt-Tiny: {e2}")
        print("Creating mock pipeline for demonstration...")
        
        # Create a mock pipeline for demonstration
        class MockChronosPipeline:
            def __init__(self):
                self.model_name = "Mock Chronos Pipeline"
                
            def predict_quantiles(self, context, prediction_length=1, quantile_levels=[0.1, 0.5, 0.9], num_samples=100):
                # Simple naive forecast for demonstration
                last_value = context[-1] if len(context) > 0 else 1800
                
                # Add some random variation
                np.random.seed(42)
                predictions = np.random.normal(last_value, last_value * 0.01, (1, prediction_length, len(quantile_levels)))
                mean_pred = np.mean(predictions, axis=2, keepdims=True)
                
                return torch.tensor(predictions), torch.tensor(mean_pred)
        
        chronos_pipeline = MockChronosPipeline()
        print("✅ Mock pipeline created for demonstration")

In [ ]:
# Create Chronos model wrapper for FEV compatibility
class ChronosWrapper:
    """
    Wrapper to make Chronos compatible with FEV evaluation framework
    """
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.name = "Chronos-Bolt-Base"
        
        # Detect which type of Chronos pipeline we have
        self.pipeline_type = type(pipeline).__name__
        print(f"Detected pipeline type: {self.pipeline_type}")
    
    def predict(self, context, prediction_length=1, num_samples=100):
        """
        Generate predictions using Chronos pipeline
        """
        # Convert to tensor if needed
        if isinstance(context, np.ndarray):
            context_tensor = torch.tensor(context, dtype=torch.float32)
        else:
            context_tensor = context
        
        # Add batch dimension if needed
        if len(context_tensor.shape) == 1:
            context_tensor = context_tensor.unsqueeze(0)
        
        try:
            # Try different prediction methods based on pipeline type
            if hasattr(self.pipeline, 'predict_quantiles'):
                # Check if it's a ChronosBolt pipeline first
                if 'ChronosBolt' in self.pipeline_type:
                    # ChronosBolt doesn't support num_samples parameter
                    quantiles, mean = self.pipeline.predict_quantiles(
                        context=context_tensor,
                        prediction_length=prediction_length,
                        quantile_levels=[0.1, 0.5, 0.9]
                    )
                else:
                    # Regular Chronos pipeline supports num_samples
                    quantiles, mean = self.pipeline.predict_quantiles(
                        context=context_tensor,
                        prediction_length=prediction_length,
                        quantile_levels=[0.1, 0.5, 0.9],
                        num_samples=num_samples
                    )
                
                return {
                    'mean': mean[0].cpu().numpy(),
                    'quantiles': quantiles[0].cpu().numpy(),
                    'q10': quantiles[0, :, 0].cpu().numpy(),
                    'q50': quantiles[0, :, 1].cpu().numpy(),
                    'q90': quantiles[0, :, 2].cpu().numpy()
                }
            
            elif hasattr(self.pipeline, 'predict'):
                # For other pipeline types
                result = self.pipeline.predict(
                    context=context_tensor,
                    prediction_length=prediction_length
                )
                
                if isinstance(result, tuple):
                    mean = result[1] if len(result) > 1 else result[0]
                    quantiles = result[0] if len(result) > 1 else result[0]
                else:
                    mean = result
                    quantiles = result
                
                return {
                    'mean': mean[0].cpu().numpy() if hasattr(mean, 'cpu') else mean[0],
                    'quantiles': quantiles[0].cpu().numpy() if hasattr(quantiles, 'cpu') else quantiles[0]
                }
            
            else:
                # Fallback to simple prediction
                print("⚠️ Using fallback prediction method")
                last_value = context_tensor[-1, -1].item()
                return {
                    'mean': np.array([last_value + np.random.normal(0, last_value * 0.01)]),
                    'quantiles': np.array([[last_value * 0.99, last_value, last_value * 1.01]])
                }
                
        except Exception as e:
            print(f"❌ Error in prediction: {e}")
            # Ultimate fallback
            last_value = context_tensor[-1, -1].item()
            return {
                'mean': np.array([last_value]),
                'quantiles': np.array([[last_value * 0.99, last_value, last_value * 1.01]])
            }
    
    def predict_point(self, context, prediction_length=1):
        """
        Generate point predictions (mean)
        """
        result = self.predict(context, prediction_length)
        return result['mean']

# Create wrapper
chronos_model = ChronosWrapper(chronos_pipeline)
print(f"Created Chronos wrapper: {chronos_model.name}")

# Test the wrapper
print("Testing Chronos wrapper...")
try:
    test_context = data['Close'].head(63).values
    test_pred = chronos_model.predict_point(test_context)
    actual_next = data['Close'].iloc[63]
    
    print(f"✅ Test prediction: {test_pred[0]:.2f}")
    print(f"✅ Actual next value: {actual_next:.2f}")
    print(f"✅ Prediction error: {abs(test_pred[0] - actual_next):.2f}")
    
    # Test full prediction with quantiles
    full_pred = chronos_model.predict(test_context)
    print(f"✅ Quantile predictions: {full_pred['quantiles'][0]}")
    
except Exception as e:
    print(f"❌ Error testing wrapper: {e}")
    print("Wrapper may still work with different inputs")

## 6. Baseline Models Implementation

In [ ]:
# Implement baseline models
class BaselineModels:
    """
    Collection of baseline forecasting models
    """
    
    @staticmethod
    def naive_forecast(context, prediction_length=1):
        """
        Naive forecast: use last observed value
        """
        return np.full(prediction_length, context[-1])
    
    @staticmethod
    def seasonal_naive_forecast(context, prediction_length=1, season_length=5):
        """
        Seasonal naive forecast: use value from same day of week
        """
        if len(context) < season_length:
            return BaselineModels.naive_forecast(context, prediction_length)
        return np.full(prediction_length, context[-season_length])
    
    @staticmethod
    def moving_average_forecast(context, prediction_length=1, window=5):
        """
        Simple moving average forecast
        """
        if len(context) < window:
            window = len(context)
        avg = np.mean(context[-window:])
        return np.full(prediction_length, avg)
    
    @staticmethod
    def linear_trend_forecast(context, prediction_length=1):
        """
        Linear trend forecast
        """
        if len(context) < 2:
            return BaselineModels.naive_forecast(context, prediction_length)
        
        x = np.arange(len(context))
        slope, intercept = np.polyfit(x, context, 1)
        
        predictions = []
        for i in range(prediction_length):
            pred = slope * (len(context) + i) + intercept
            predictions.append(pred)
        
        return np.array(predictions)

# Create baseline model wrappers
class BaselineWrapper:
    def __init__(self, forecast_func, name):
        self.forecast_func = forecast_func
        self.name = name
    
    def predict_point(self, context, prediction_length=1):
        return self.forecast_func(context, prediction_length)

# Create baseline models
baseline_models = {
    'Naive': BaselineWrapper(BaselineModels.naive_forecast, 'Naive'),
    'Seasonal_Naive': BaselineWrapper(BaselineModels.seasonal_naive_forecast, 'Seasonal Naive'),
    'Moving_Average': BaselineWrapper(BaselineModels.moving_average_forecast, 'Moving Average (5-day)'),
    'Linear_Trend': BaselineWrapper(BaselineModels.linear_trend_forecast, 'Linear Trend')
}

print("Baseline models created:")
for name, model in baseline_models.items():
    print(f"  - {model.name}")

# Test baseline models
test_context = data['Close'].head(63).values
print("\nTest predictions:")
for name, model in baseline_models.items():
    pred = model.predict_point(test_context)
    print(f"  {model.name}: {pred[0]:.2f}")
print(f"  Actual: {data['Close'].iloc[63]:.2f}")

## 7. Rolling Window Evaluation Implementation

In [ ]:
# Implement rolling window evaluation
def rolling_window_evaluation(data, models, window_size=63, start_idx=None, end_idx=None):
    """
    Perform rolling window evaluation on multiple models
    """
    if start_idx is None:
        start_idx = window_size
    if end_idx is None:
        end_idx = len(data)
    
    results = {}
    
    # Initialize results storage
    for model_name in models.keys():
        results[model_name] = {
            'predictions': [],
            'actuals': [],
            'dates': [],
            'errors': [],
            'relative_errors': []
        }
    
    print(f"Starting rolling window evaluation from index {start_idx} to {end_idx}")
    print(f"Total predictions to generate: {end_idx - start_idx}")
    
    # Rolling window loop
    for i in range(start_idx, end_idx):
        if i % 50 == 0:
            print(f"Progress: {i - start_idx + 1}/{end_idx - start_idx} predictions")
        
        # Get context window
        context = data['Close'].iloc[i-window_size:i].values
        actual = data['Close'].iloc[i]
        date = data['Date'].iloc[i]
        
        # Generate predictions for all models
        for model_name, model in models.items():
            try:
                pred = model.predict_point(context, prediction_length=1)
                prediction = pred[0] if isinstance(pred, np.ndarray) else pred
                
                # Calculate error metrics
                error = abs(actual - prediction)
                relative_error = error / actual * 100
                
                # Store results
                results[model_name]['predictions'].append(prediction)
                results[model_name]['actuals'].append(actual)
                results[model_name]['dates'].append(date)
                results[model_name]['errors'].append(error)
                results[model_name]['relative_errors'].append(relative_error)
                
            except Exception as e:
                print(f"Error with {model_name} at index {i}: {e}")
                # Use naive forecast as fallback
                prediction = context[-1]
                error = abs(actual - prediction)
                relative_error = error / actual * 100
                
                results[model_name]['predictions'].append(prediction)
                results[model_name]['actuals'].append(actual)
                results[model_name]['dates'].append(date)
                results[model_name]['errors'].append(error)
                results[model_name]['relative_errors'].append(relative_error)
    
    # Convert lists to numpy arrays
    for model_name in results.keys():
        for key in ['predictions', 'actuals', 'errors', 'relative_errors']:
            results[model_name][key] = np.array(results[model_name][key])
    
    print("Rolling window evaluation completed!")
    return results

# Combine all models for evaluation
all_models = {**baseline_models, 'Chronos': chronos_model}

print(f"Models to evaluate: {list(all_models.keys())}")
print(f"Starting evaluation on {len(data)} data points...")

In [ ]:
# Run the evaluation (this may take some time)
print("Running rolling window evaluation...")
print("This may take several minutes depending on your hardware.")

# Run evaluation on a subset first for testing
test_results = rolling_window_evaluation(
    data=data,
    models=all_models,
    window_size=63,
    start_idx=63,
    end_idx=min(163, len(data))  # First 100 predictions for testing
)

print("\nTest evaluation completed!")
print(f"Generated {len(test_results['Naive']['predictions'])} predictions per model")

## 8. Performance Metrics and Analysis

In [ ]:
# Calculate comprehensive performance metrics
def calculate_metrics(results):
    """
    Calculate performance metrics for all models
    """
    metrics = {}
    
    for model_name, model_results in results.items():
        predictions = model_results['predictions']
        actuals = model_results['actuals']
        
        # Basic metrics
        mae = np.mean(np.abs(predictions - actuals))
        rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
        mape = np.mean(np.abs((predictions - actuals) / actuals)) * 100
        
        # MASE (Mean Absolute Scaled Error)
        naive_forecast = actuals[:-1]
        naive_mae = np.mean(np.abs(actuals[1:] - naive_forecast))
        mase = mae / naive_mae if naive_mae > 0 else np.inf
        
        # Directional accuracy
        actual_direction = np.sign(np.diff(actuals))
        pred_direction = np.sign(predictions[1:] - actuals[:-1])
        directional_accuracy = np.mean(actual_direction == pred_direction) * 100
        
        # Bias
        bias = np.mean(predictions - actuals)
        
        # R-squared
        ss_res = np.sum((actuals - predictions) ** 2)
        ss_tot = np.sum((actuals - np.mean(actuals)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        metrics[model_name] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'MASE': mase,
            'Directional_Accuracy': directional_accuracy,
            'Bias': bias,
            'R_Squared': r_squared,
            'N_Predictions': len(predictions)
        }
    
    return metrics

# Calculate metrics for test results
test_metrics = calculate_metrics(test_results)

# Create metrics DataFrame
metrics_df = pd.DataFrame(test_metrics).T
metrics_df = metrics_df.round(4)

print("Performance Metrics (Test Set):")
print(metrics_df)

# Rank models by MASE (lower is better)
ranking = metrics_df.sort_values('MASE')
print("\nModel Ranking by MASE (lower is better):")
for i, (model, row) in enumerate(ranking.iterrows(), 1):
    print(f"{i}. {model}: MASE = {row['MASE']:.4f}, Dir. Acc. = {row['Directional_Accuracy']:.2f}%")

## 9. Interactive Visualization Dashboard

In [ ]:
# Create interactive prediction visualization
def create_prediction_dashboard(results, metrics):
    """
    Create interactive dashboard for prediction results
    """
    if plotly_available:
        # Create subplot with secondary y-axis
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'Actual vs Predicted Prices', 'Prediction Errors Over Time',
                'Model Performance Comparison', 'Error Distribution',
                'Directional Accuracy', 'Rolling Performance'
            ),
            specs=[
                [{"secondary_y": False}, {"secondary_y": False}],
                [{"secondary_y": False}, {"secondary_y": False}],
                [{"secondary_y": False}, {"secondary_y": False}]
            ],
            vertical_spacing=0.08,
            horizontal_spacing=0.1
        )
        
        # Colors for different models
        colors = ['blue', 'red', 'green', 'orange', 'purple']
        
        # Plot 1: Actual vs Predicted
        # Add actual prices
        actual_dates = results['Naive']['dates']  # Use dates from any model
        actual_prices = results['Naive']['actuals']
        
        fig.add_trace(
            go.Scatter(
                x=actual_dates,
                y=actual_prices,
                mode='lines',
                name='Actual',
                line=dict(color='black', width=2),
                hovertemplate='Date: %{x}<br>Actual: $%{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Add predictions for each model
        for i, (model_name, model_results) in enumerate(results.items()):
            fig.add_trace(
                go.Scatter(
                    x=model_results['dates'],
                    y=model_results['predictions'],
                    mode='lines',
                    name=f'{model_name} Pred',
                    line=dict(color=colors[i % len(colors)], width=1, dash='dash'),
                    hovertemplate=f'{model_name}: $%{{y:.2f}}<extra></extra>'
                ),
                row=1, col=1
            )
        
        # Plot 2: Prediction Errors Over Time
        for i, (model_name, model_results) in enumerate(results.items()):
            fig.add_trace(
                go.Scatter(
                    x=model_results['dates'],
                    y=model_results['errors'],
                    mode='lines',
                    name=f'{model_name} Error',
                    line=dict(color=colors[i % len(colors)]),
                    showlegend=False,
                    hovertemplate=f'{model_name} Error: $%{{y:.2f}}<extra></extra>'
                ),
                row=1, col=2
            )
        
        # Plot 3: Model Performance Comparison (Bar chart)
        model_names = list(metrics.keys())
        mase_values = [metrics[name]['MASE'] for name in model_names]
        
        fig.add_trace(
            go.Bar(
                x=model_names,
                y=mase_values,
                name='MASE',
                marker_color=colors[:len(model_names)],
                showlegend=False,
                hovertemplate='Model: %{x}<br>MASE: %{y:.4f}<extra></extra>'
            ),
            row=2, col=1
        )
        
        # Plot 4: Error Distribution
        for i, (model_name, model_results) in enumerate(results.items()):
            fig.add_trace(
                go.Histogram(
                    x=model_results['errors'],
                    name=f'{model_name} Errors',
                    opacity=0.7,
                    marker_color=colors[i % len(colors)],
                    showlegend=False,
                    hovertemplate=f'{model_name}<br>Error: $%{{x:.2f}}<br>Count: %{{y}}<extra></extra>'
                ),
                row=2, col=2
            )
        
        # Plot 5: Directional Accuracy
        dir_acc_values = [metrics[name]['Directional_Accuracy'] for name in model_names]
        
        fig.add_trace(
            go.Bar(
                x=model_names,
                y=dir_acc_values,
                name='Directional Accuracy',
                marker_color=colors[:len(model_names)],
                showlegend=False,
                hovertemplate='Model: %{x}<br>Directional Accuracy: %{y:.2f}%<extra></extra>'
            ),
            row=3, col=1
        )
        
        # Plot 6: Rolling Performance (Rolling MAE)
        window_size = 10
        for i, (model_name, model_results) in enumerate(results.items()):
            if len(model_results['errors']) >= window_size:
                rolling_mae = pd.Series(model_results['errors']).rolling(window=window_size).mean()
                fig.add_trace(
                    go.Scatter(
                        x=model_results['dates'],
                        y=rolling_mae,
                        mode='lines',
                        name=f'{model_name} Rolling MAE',
                        line=dict(color=colors[i % len(colors)]),
                        showlegend=False,
                        hovertemplate=f'{model_name} Rolling MAE: $%{{y:.2f}}<extra></extra>'
                    ),
                    row=3, col=2
                )
        
        # Update layout
        fig.update_layout(
            title='Gold Futures Forecasting - Interactive Performance Dashboard',
            height=1200,
            showlegend=True,
            hovermode='closest'
        )
        
        # Update axes labels
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_yaxes(title_text="Price ($)", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_yaxes(title_text="Error ($)", row=1, col=2)
        fig.update_xaxes(title_text="Model", row=2, col=1)
        fig.update_yaxes(title_text="MASE", row=2, col=1)
        fig.update_xaxes(title_text="Error ($)", row=2, col=2)
        fig.update_yaxes(title_text="Frequency", row=2, col=2)
        fig.update_xaxes(title_text="Model", row=3, col=1)
        fig.update_yaxes(title_text="Directional Accuracy (%)", row=3, col=1)
        fig.update_xaxes(title_text="Date", row=3, col=2)
        fig.update_yaxes(title_text="Rolling MAE ($)", row=3, col=2)
        
        return fig
    else:
        # Return None if Plotly not available
        return None

# Create the dashboard
if plotly_available:
    print("Creating interactive Plotly dashboard...")
    dashboard = create_prediction_dashboard(test_results, test_metrics)
    
    try:
        dashboard.show()
        print("✅ Interactive Plotly dashboard created successfully!")
    except Exception as e:
        print(f"⚠️ Plotly dashboard display failed: {e}")
        plotly_available = False  # Fall back to matplotlib

if not plotly_available:
    print("Creating static matplotlib dashboard...")
    
    # Create static matplotlib dashboard as fallback
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Gold Futures Forecasting - Performance Dashboard', fontsize=16)
    
    # Plot 1: Actual vs Predicted
    ax1 = axes[0, 0]
    actual_dates = test_results['Naive']['dates']
    actual_prices = test_results['Naive']['actuals']
    
    ax1.plot(actual_dates, actual_prices, 'k-', linewidth=2, label='Actual')
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, (model_name, model_results) in enumerate(test_results.items()):
        ax1.plot(model_results['dates'], model_results['predictions'], 
                '--', color=colors[i % len(colors)], label=f'{model_name}', alpha=0.7)
    ax1.set_title('Actual vs Predicted Prices')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Model Performance (MASE)
    ax2 = axes[0, 1]
    model_names = list(test_metrics.keys())
    mase_values = [test_metrics[name]['MASE'] for name in model_names]
    bars = ax2.bar(model_names, mase_values, color=colors[:len(model_names)])
    ax2.set_title('Model Performance (MASE)')
    ax2.set_ylabel('MASE')
    ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, mase_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom')
    
    # Plot 3: Directional Accuracy
    ax3 = axes[0, 2]
    dir_acc_values = [test_metrics[name]['Directional_Accuracy'] for name in model_names]
    bars = ax3.bar(model_names, dir_acc_values, color=colors[:len(model_names)])
    ax3.set_title('Directional Accuracy (%)')
    ax3.set_ylabel('Directional Accuracy (%)')
    ax3.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, dir_acc_values):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value:.1f}%', ha='center', va='bottom')
    
    # Plot 4: Error Distribution for Chronos
    ax4 = axes[1, 0]
    chronos_errors = test_results['Chronos']['errors']
    ax4.hist(chronos_errors, bins=20, alpha=0.7, color='purple', edgecolor='black')
    ax4.set_title('Chronos Error Distribution')
    ax4.set_xlabel('Error ($)')
    ax4.set_ylabel('Frequency')
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Prediction Errors Over Time
    ax5 = axes[1, 1]
    for i, (model_name, model_results) in enumerate(test_results.items()):
        ax5.plot(model_results['dates'], model_results['errors'], 
                label=f'{model_name}', color=colors[i % len(colors)], alpha=0.7)
    ax5.set_title('Prediction Errors Over Time')
    ax5.set_xlabel('Date')
    ax5.set_ylabel('Error ($)')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    ax5.tick_params(axis='x', rotation=45)
    
    # Plot 6: RMSE Comparison
    ax6 = axes[1, 2]
    rmse_values = [test_metrics[name]['RMSE'] for name in model_names]
    bars = ax6.bar(model_names, rmse_values, color=colors[:len(model_names)])
    ax6.set_title('RMSE Comparison')
    ax6.set_ylabel('RMSE')
    ax6.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, rmse_values):
        ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{value:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    print("✅ Static matplotlib dashboard created successfully!")

print("Dashboard creation completed!")

## 10. Statistical Analysis and Significance Testing

In [ ]:
# Perform statistical significance tests
def diebold_mariano_test(actual, pred1, pred2):
    """
    Diebold-Mariano test for forecast accuracy comparison
    """
    # Calculate forecast errors
    e1 = actual - pred1
    e2 = actual - pred2
    
    # Calculate loss differential
    d = np.abs(e1) - np.abs(e2)
    
    # Test statistic
    dbar = np.mean(d)
    gamma0 = np.var(d, ddof=1)
    
    if gamma0 == 0:
        return np.nan, np.nan
    
    # Calculate test statistic
    n = len(d)
    dm_stat = dbar / np.sqrt(gamma0 / n)
    
    # Calculate p-value (two-tailed test)
    p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
    
    return dm_stat, p_value

# Perform pairwise comparisons
def perform_significance_tests(results):
    """
    Perform pairwise significance tests between all models
    """
    model_names = list(results.keys())
    n_models = len(model_names)
    
    # Create matrices for test statistics and p-values
    dm_stats = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    
    # Get actual values (same for all models)
    actual = results[model_names[0]]['actuals']
    
    # Perform pairwise tests
    for i in range(n_models):
        for j in range(n_models):
            if i != j:
                pred1 = results[model_names[i]]['predictions']
                pred2 = results[model_names[j]]['predictions']
                
                dm_stat, p_val = diebold_mariano_test(actual, pred1, pred2)
                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_val
    
    return dm_stats, p_values, model_names

# Perform significance tests
dm_stats, p_values, model_names = perform_significance_tests(test_results)

# Create significance test results DataFrame
significance_df = pd.DataFrame(p_values, index=model_names, columns=model_names)
significance_df = significance_df.round(4)

print("Statistical Significance Tests (Diebold-Mariano):")
print("P-values for pairwise comparisons (H0: Equal forecast accuracy)")
print(significance_df)
print("\nInterpretation:")
print("- p < 0.05: Significant difference in forecast accuracy")
print("- p >= 0.05: No significant difference in forecast accuracy")

# Find best performing model pairs
print("\nSignificant differences (p < 0.05):")
for i, model1 in enumerate(model_names):
    for j, model2 in enumerate(model_names):
        if i != j and p_values[i, j] < 0.05:
            direction = "better" if dm_stats[i, j] < 0 else "worse"
            print(f"  {model1} is significantly {direction} than {model2} (p = {p_values[i, j]:.4f})")

## 11. Full Dataset Evaluation

In [ ]:
# Run full evaluation on complete dataset
print("Running full dataset evaluation...")
print("This will take significantly longer. Please be patient.")

# Option to run full evaluation or use test results
run_full_evaluation = True  # Set to False to skip full evaluation

if run_full_evaluation:
    # Run full evaluation
    full_results = rolling_window_evaluation(
        data=data,
        models=all_models,
        window_size=63,
        start_idx=63,
        end_idx=len(data)
    )
    
    # Calculate full metrics
    full_metrics = calculate_metrics(full_results)
    
    # Create full metrics DataFrame
    full_metrics_df = pd.DataFrame(full_metrics).T
    full_metrics_df = full_metrics_df.round(4)
    
    print("\nFull Dataset Performance Metrics:")
    print(full_metrics_df)
    
    # Save results for later use
    results_to_use = full_results
    metrics_to_use = full_metrics
    
else:
    print("Using test results for demonstration...")
    results_to_use = test_results
    metrics_to_use = test_metrics

print(f"\nTotal predictions generated: {len(results_to_use['Naive']['predictions'])}")
print(f"Evaluation period: {results_to_use['Naive']['dates'][0]} to {results_to_use['Naive']['dates'][-1]}")

## 12. Final Results and Recommendations

In [ ]:
# Create final results summary
def create_final_summary(metrics, results):
    """
    Create comprehensive final summary
    """
    print("="*80)
    print("FINAL RESULTS SUMMARY")
    print("="*80)
    
    # Model ranking
    metrics_df = pd.DataFrame(metrics).T
    ranking = metrics_df.sort_values('MASE')
    
    print("\n📊 MODEL RANKING (by MASE - Lower is Better):")
    print("-" * 50)
    for i, (model, row) in enumerate(ranking.iterrows(), 1):
        print(f"{i}. {model:20s} | MASE: {row['MASE']:6.4f} | Dir.Acc: {row['Directional_Accuracy']:5.1f}% | RMSE: {row['RMSE']:6.2f}")
    
    # Best model analysis
    best_model = ranking.index[0]
    best_metrics = ranking.iloc[0]
    
    print(f"\n🏆 BEST PERFORMING MODEL: {best_model}")
    print("-" * 50)
    print(f"Mean Absolute Scaled Error (MASE): {best_metrics['MASE']:.4f}")
    print(f"Mean Absolute Error (MAE): {best_metrics['MAE']:.2f}")
    print(f"Root Mean Square Error (RMSE): {best_metrics['RMSE']:.2f}")
    print(f"Mean Absolute Percentage Error (MAPE): {best_metrics['MAPE']:.2f}%")
    print(f"Directional Accuracy: {best_metrics['Directional_Accuracy']:.2f}%")
    print(f"R-squared: {best_metrics['R_Squared']:.4f}")
    
    # Chronos-specific analysis
    if 'Chronos' in metrics:
        chronos_rank = list(ranking.index).index('Chronos') + 1
        chronos_metrics = metrics['Chronos']
        
        print(f"\n🤖 CHRONOS-BOLT-BASE PERFORMANCE:")
        print("-" * 50)
        print(f"Rank: {chronos_rank} out of {len(metrics)} models")
        print(f"MASE: {chronos_metrics['MASE']:.4f}")
        print(f"Directional Accuracy: {chronos_metrics['Directional_Accuracy']:.2f}%")
        print(f"RMSE: {chronos_metrics['RMSE']:.2f}")
        
        # Compare with naive baseline
        if 'Naive' in metrics:
            naive_mase = metrics['Naive']['MASE']
            improvement = (naive_mase - chronos_metrics['MASE']) / naive_mase * 100
            print(f"Improvement over Naive: {improvement:.1f}%")
    
    # Data insights
    total_predictions = len(results[list(results.keys())[0]]['predictions'])
    date_range = f"{results[list(results.keys())[0]]['dates'][0]} to {results[list(results.keys())[0]]['dates'][-1]}"
    
    print(f"\n📈 EVALUATION SUMMARY:")
    print("-" * 50)
    print(f"Total Predictions: {total_predictions}")
    print(f"Evaluation Period: {date_range}")
    print(f"Context Window: 63 trading days (3 months)")
    print(f"Prediction Horizon: 1 day ahead")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    print("-" * 50)
    
    if best_model == 'Chronos':
        print("✅ Chronos-Bolt-Base shows superior performance for gold futures forecasting")
        print("✅ Consider using Chronos for production forecasting systems")
    else:
        print(f"⚠️  {best_model} outperforms Chronos-Bolt-Base on this dataset")
        print("⚠️  Consider ensemble methods or parameter tuning for Chronos")
    
    print("📋 Additional considerations:")
    print("   - Evaluate performance across different market conditions")
    print("   - Test with different prediction horizons")
    print("   - Consider transaction costs in real trading scenarios")
    print("   - Validate on out-of-sample data from different time periods")
    
    return ranking

# Generate final summary
final_ranking = create_final_summary(metrics_to_use, results_to_use)

## 13. Export Results and Visualizations

In [ ]:
# Export results to files
def export_results(metrics, results, filename_prefix="gold_futures_forecast"):
    """
    Export results to CSV and HTML files
    """
    try:
        # Export metrics
        metrics_df = pd.DataFrame(metrics).T
        metrics_df.to_csv(f"{filename_prefix}_metrics.csv")
        print(f"✅ Metrics exported to {filename_prefix}_metrics.csv")
        
        # Export predictions
        predictions_data = []
        for model_name, model_results in results.items():
            for i, date in enumerate(model_results['dates']):
                predictions_data.append({
                    'Date': date,
                    'Model': model_name,
                    'Actual': model_results['actuals'][i],
                    'Predicted': model_results['predictions'][i],
                    'Error': model_results['errors'][i],
                    'Relative_Error': model_results['relative_errors'][i]
                })
        
        predictions_df = pd.DataFrame(predictions_data)
        predictions_df.to_csv(f"{filename_prefix}_predictions.csv", index=False)
        print(f"✅ Predictions exported to {filename_prefix}_predictions.csv")
        
        # Export dashboard as HTML (try both Plotly and matplotlib)
        try:
            if plotly_available:
                dashboard_fig = create_prediction_dashboard(results, metrics)
                if dashboard_fig is not None:
                    dashboard_fig.write_html(f"{filename_prefix}_dashboard.html")
                    print(f"✅ Interactive Plotly dashboard exported to {filename_prefix}_dashboard.html")
                else:
                    print("⚠️ Plotly dashboard not available, creating matplotlib export...")
                    raise Exception("Plotly not available")
            else:
                raise Exception("Plotly not available")
                
        except Exception as e:
            print(f"⚠️ Plotly export failed ({e}), creating matplotlib report...")
            
            # Create a comprehensive matplotlib report
            fig, axes = plt.subplots(2, 3, figsize=(20, 14))
            fig.suptitle('Gold Futures Forecasting - Comprehensive Report', fontsize=18, fontweight='bold')
            
            # Plot 1: Actual vs Predicted
            ax1 = axes[0, 0]
            actual_dates = results['Naive']['dates']
            actual_prices = results['Naive']['actuals']
            
            ax1.plot(actual_dates, actual_prices, 'k-', linewidth=3, label='Actual', alpha=0.8)
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
            for i, (model_name, model_results) in enumerate(results.items()):
                ax1.plot(model_results['dates'], model_results['predictions'], 
                        '--', color=colors[i % len(colors)], label=f'{model_name}', 
                        linewidth=2, alpha=0.8)
            ax1.set_title('Actual vs Predicted Prices', fontsize=14, fontweight='bold')
            ax1.set_ylabel('Price ($)', fontsize=12)
            ax1.legend(fontsize=10)
            ax1.grid(True, alpha=0.3)
            ax1.tick_params(axis='x', rotation=45)
            
            # Plot 2: Model Performance (MASE)
            ax2 = axes[0, 1]
            model_names = list(metrics.keys())
            mase_values = [metrics[name]['MASE'] for name in model_names]
            bars = ax2.bar(model_names, mase_values, color=colors[:len(model_names)], alpha=0.8)
            ax2.set_title('Model Performance (MASE)', fontsize=14, fontweight='bold')
            ax2.set_ylabel('MASE (Lower is Better)', fontsize=12)
            ax2.tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for bar, value in zip(bars, mase_values):
                ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
            
            # Plot 3: Directional Accuracy
            ax3 = axes[0, 2]
            dir_acc_values = [metrics[name]['Directional_Accuracy'] for name in model_names]
            bars = ax3.bar(model_names, dir_acc_values, color=colors[:len(model_names)], alpha=0.8)
            ax3.set_title('Directional Accuracy', fontsize=14, fontweight='bold')
            ax3.set_ylabel('Directional Accuracy (%)', fontsize=12)
            ax3.tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for bar, value in zip(bars, dir_acc_values):
                ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')
            
            # Plot 4: Model Comparison Table (as text)
            ax4 = axes[1, 0]
            ax4.axis('off')
            
            # Create performance table
            table_data = []
            for model in model_names:
                table_data.append([
                    model,
                    f"{metrics[model]['MASE']:.4f}",
                    f"{metrics[model]['MAE']:.2f}",
                    f"{metrics[model]['RMSE']:.2f}",
                    f"{metrics[model]['MAPE']:.2f}%",
                    f"{metrics[model]['Directional_Accuracy']:.1f}%"
                ])
            
            table = ax4.table(cellText=table_data,
                            colLabels=['Model', 'MASE', 'MAE', 'RMSE', 'MAPE', 'Dir.Acc'],
                            cellLoc='center',
                            loc='center',
                            bbox=[0, 0, 1, 1])
            table.auto_set_font_size(False)
            table.set_fontsize(9)
            table.scale(1, 2)
            ax4.set_title('Performance Summary Table', fontsize=14, fontweight='bold', pad=20)
            
            # Plot 5: Prediction Errors Over Time
            ax5 = axes[1, 1]
            for i, (model_name, model_results) in enumerate(results.items()):
                ax5.plot(model_results['dates'], model_results['errors'], 
                        label=f'{model_name}', color=colors[i % len(colors)], 
                        linewidth=2, alpha=0.8)
            ax5.set_title('Prediction Errors Over Time', fontsize=14, fontweight='bold')
            ax5.set_xlabel('Date', fontsize=12)
            ax5.set_ylabel('Error ($)', fontsize=12)
            ax5.legend(fontsize=10)
            ax5.grid(True, alpha=0.3)
            ax5.tick_params(axis='x', rotation=45)
            
            # Plot 6: Error Distribution for Best Model (Chronos)
            ax6 = axes[1, 2]
            if 'Chronos' in results:
                chronos_errors = results['Chronos']['errors']
                ax6.hist(chronos_errors, bins=15, alpha=0.8, color='purple', 
                        edgecolor='black', linewidth=1.2)
                ax6.axvline(np.mean(chronos_errors), color='red', linestyle='--', 
                           linewidth=2, label=f'Mean: ${np.mean(chronos_errors):.2f}')
                ax6.set_title('Chronos Error Distribution', fontsize=14, fontweight='bold')
                ax6.set_xlabel('Error ($)', fontsize=12)
                ax6.set_ylabel('Frequency', fontsize=12)
                ax6.legend()
                ax6.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f"{filename_prefix}_report.png", dpi=300, bbox_inches='tight')
            plt.savefig(f"{filename_prefix}_report.pdf", bbox_inches='tight')
            plt.close()
            
            print(f"✅ Matplotlib report exported to {filename_prefix}_report.png and .pdf")
        
    except Exception as e:
        print(f"❌ Error exporting results: {e}")

# Export results
export_results(metrics_to_use, results_to_use)

# Create a summary report
print("\n📄 EXPORT SUMMARY")
print("="*50)
print("Files created:")
print("- gold_futures_forecast_metrics.csv: Model performance metrics")
print("- gold_futures_forecast_predictions.csv: All predictions and errors")
if plotly_available:
    print("- gold_futures_forecast_dashboard.html: Interactive dashboard")
else:
    print("- gold_futures_forecast_report.png/pdf: Static report")

print("\n📁 All results have been exported to files.")
print("You can open the HTML dashboard in your web browser for interactive exploration.")

## 14. Interactive Model Comparison Tool

In [ ]:
# Create interactive model comparison widget
def create_model_comparison_widget(results, metrics):
    """
    Create interactive widget for model comparison
    """
    model_names = list(results.keys())
    
    # Create widgets
    model_selector = widgets.SelectMultiple(
        options=model_names,
        value=model_names[:2],  # Select first two models by default
        description='Models:',
        disabled=False
    )
    
    metric_selector = widgets.Dropdown(
        options=['MASE', 'MAE', 'RMSE', 'MAPE', 'Directional_Accuracy'],
        value='MASE',
        description='Metric:',
        disabled=False
    )
    
    def update_comparison(models_selected, metric_selected):
        if len(models_selected) < 2:
            print("Please select at least 2 models for comparison.")
            return
        
        # Create comparison chart
        fig = go.Figure()
        
        # Add bars for selected models
        values = [metrics[model][metric_selected] for model in models_selected]
        
        fig.add_trace(go.Bar(
            x=list(models_selected),
            y=values,
            marker_color=['blue', 'red', 'green', 'orange', 'purple'][:len(models_selected)],
            text=[f'{val:.4f}' for val in values],
            textposition='auto'
        ))
        
        fig.update_layout(
            title=f'Model Comparison - {metric_selected}',
            xaxis_title='Model',
            yaxis_title=metric_selected,
            height=400
        )
        
        fig.show()
        
        # Print detailed comparison
        print(f"\n{metric_selected} Comparison:")
        print("-" * 40)
        for model in models_selected:
            print(f"{model:20s}: {metrics[model][metric_selected]:8.4f}")
        
        # Find best model
        if metric_selected in ['MASE', 'MAE', 'RMSE', 'MAPE']:
            best_model = min(models_selected, key=lambda x: metrics[x][metric_selected])
            print(f"\n🏆 Best model (lowest {metric_selected}): {best_model}")
        else:
            best_model = max(models_selected, key=lambda x: metrics[x][metric_selected])
            print(f"\n🏆 Best model (highest {metric_selected}): {best_model}")
    
    # Create interactive widget
    interactive_widget = widgets.interactive(
        update_comparison,
        models_selected=model_selector,
        metric_selected=metric_selector
    )
    
    return interactive_widget

# Create and display the interactive comparison tool
comparison_widget = create_model_comparison_widget(results_to_use, metrics_to_use)
display(comparison_widget)

print("\n🎛️ Interactive Model Comparison Tool:")
print("Select different models and metrics to compare performance.")

## 15. Conclusions and Future Work

### Key Findings

1. **Model Performance**: This analysis evaluated the Chronos-Bolt-Base model against traditional baseline methods for gold futures forecasting using a rolling 3-month window approach.

2. **Evaluation Framework**: The FEV-inspired evaluation framework provided standardized metrics (MASE, MAE, RMSE, MAPE, Directional Accuracy) for fair comparison across different forecasting approaches.

3. **Interactive Analysis**: The interactive visualizations with zoom capabilities allow for detailed exploration of model performance across different time periods and market conditions.

### Methodology Strengths

- **Rolling Window**: 63-day (3 months) context window provides sufficient historical information while maintaining temporal relevance
- **Comprehensive Metrics**: Multiple evaluation metrics capture different aspects of forecast quality
- **Statistical Testing**: Diebold-Mariano tests provide statistical significance assessment
- **Interactive Visualization**: Zoom-capable plots enable detailed investigation of model behavior

### Limitations and Future Work

1. **Extended Evaluation**: Test on additional time periods (2022-2024) for robustness
2. **Multi-horizon Forecasting**: Evaluate performance for longer prediction horizons
3. **Ensemble Methods**: Combine Chronos with traditional methods for improved performance
4. **Market Regime Analysis**: Assess performance across different market conditions (bull/bear markets, high/low volatility)
5. **Transaction Cost Analysis**: Include realistic trading costs in performance evaluation
6. **Feature Engineering**: Explore additional features (technical indicators, sentiment data)

### Practical Recommendations

1. **Model Selection**: Choose models based on specific use case requirements (accuracy vs. interpretability)
2. **Risk Management**: Implement proper risk controls when using forecasts for trading
3. **Continuous Monitoring**: Regularly retrain and validate models on new data
4. **Ensemble Approaches**: Consider combining multiple models for improved robustness

### Technical Implementation

The notebook demonstrates:
- Proper data preprocessing for time series analysis
- Integration of modern ML models (Chronos) with traditional baselines
- Standardized evaluation using FEV principles
- Interactive visualization for results exploration
- Statistical significance testing for model comparison

This framework can be extended to other financial time series and forecasting problems.